In [2]:
import pandas as pd

df = pd.read_csv('BenignTraffic_Flows.csv')

df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.137.41-157.249.81.141-51746-80-6,192.168.137.41,51746,157.249.81.141,80,6,07/10/2022 07:15:01 PM,291959,5,4,...,32,0,0,0,0,0,0,0,0,NeedManualLabel
1,192.168.137.41-157.249.81.141-50096-443-6,192.168.137.41,50096,157.249.81.141,443,6,07/10/2022 07:15:01 PM,291320,5,4,...,32,0,0,0,0,0,0,0,0,NeedManualLabel
2,192.168.137.41-157.249.81.141-51749-80-6,192.168.137.41,51749,157.249.81.141,80,6,07/10/2022 07:15:03 PM,292739,5,4,...,32,0,0,0,0,0,0,0,0,NeedManualLabel
3,192.168.137.41-157.249.81.141-50099-443-6,192.168.137.41,50099,157.249.81.141,443,6,07/10/2022 07:15:03 PM,292398,5,4,...,32,0,0,0,0,0,0,0,0,NeedManualLabel
4,192.168.137.41-157.249.81.141-51752-80-6,192.168.137.41,51752,157.249.81.141,80,6,07/10/2022 07:15:04 PM,293252,5,4,...,32,0,0,0,0,0,0,0,0,NeedManualLabel


In [3]:
conteo = df['Flow ID'].value_counts()

conteo

8.6.0.1-8.0.6.4-0-0-0                             6481
192.168.137.61-192.168.137.1-8009-34435-6         4902
192.168.137.61-192.168.137.128-8009-39024-6       4901
192.168.137.61-192.168.137.1-8009-34287-6         4901
192.168.137.46-35.185.101.66-48335-443-6          4688
                                                  ... 
192.168.137.172-192.168.137.128-60000-55789-17       1
192.168.137.227-8.8.4.4-58518-53-17                  1
192.168.137.227-8.8.4.4-43473-53-17                  1
192.168.137.41-192.168.137.1-39669-53-17             1
192.168.137.39-47.254.14.172-38678-443-6             1
Name: Flow ID, Length: 122954, dtype: int64

In [4]:
timestamps = df[df['Flow ID'] == '192.168.137.61-192.168.137.1-8009-34435-6']['Timestamp']

In [5]:
timestamps.head()

10     07/10/2022 07:15:01 PM
38     07/10/2022 07:15:06 PM
67     07/10/2022 07:15:11 PM
94     07/10/2022 07:15:16 PM
123    07/10/2022 07:15:21 PM
Name: Timestamp, dtype: object

## Implementación Algoritmo

In [6]:
import pandas as pd
from datetime import timedelta

ventanas = {}

# Convertir el Timestamp a datetime
df["Timestamp"] = pd.to_datetime(df["Timestamp"])  # Asegurarse de que el timestamp sea datetime

for index, flujo in df.iterrows():
    ts_inicio = flujo["Timestamp"]
    duracion_us = flujo["Flow Duration"]
    ts_fin = ts_inicio + timedelta(microseconds=duracion_us)

    # Convertir a segundos desde época (float)
    ts_inicio_seg = ts_inicio.timestamp()
    ts_fin_seg = ts_fin.timestamp()

    ventana_size = 5  # tamaño de ventana en segundos
    epsilon = 1e-9

    # Calcular primera y última ventana (en segundos desde epoch)
    primera_ventana_seg = int(ts_inicio_seg // ventana_size) * ventana_size
    ultima_ventana_seg = int((ts_fin_seg - epsilon) // ventana_size) * ventana_size

    # Iterar sobre todas las ventanas afectadas por ese flujo
    ventana_actual = primera_ventana_seg
    while ventana_actual <= ultima_ventana_seg:
        inicio_ventana = ventana_actual
        fin_ventana = inicio_ventana + ventana_size
        
        # Convertimos la ventana actual a datetime para usarla de clave
        ventana_key = pd.to_datetime(inicio_ventana, unit='s')
        
        # Chequeamos superposición del flujo con esa ventana
        if ts_inicio_seg < fin_ventana and ts_fin_seg > inicio_ventana:
            if ventana_key not in ventanas:
                ventanas[ventana_key] = []
            ventanas[ventana_key].append(flujo)
        
        ventana_actual += ventana_size

# Si querés verlas ordenadas (lista de tuplas)
ventanas_ordenadas = sorted(ventanas.items(), key=lambda x: x[0])

# Ejemplo de cómo imprimir las primeras 5 ventanas con su lista de flujos
for i, (ventana, flujos) in enumerate(ventanas_ordenadas[:5]):
    print(f"{i+1}. Ventana: {ventana} → {len(flujos)} flujos")

1. Ventana: 2022-07-10 19:15:00 → 71 flujos
2. Ventana: 2022-07-10 19:15:05 → 61 flujos
3. Ventana: 2022-07-10 19:15:10 → 62 flujos
4. Ventana: 2022-07-10 19:15:15 → 48 flujos
5. Ventana: 2022-07-10 19:15:20 → 63 flujos


In [7]:
import pandas as pd
from datetime import timedelta

def procesar_flujos_por_ventanas(
    input_csv, # CSV generado por CICFlowMeter
    label,     # Etiqueta string con el tipo de tráfico del csv original
    ventana_size=5, # Tamaño de la ventana en segundos            
    output_csv="flujos_labeled.csv" # Nombre del csv resultante
):
    """
    Lee un archivo CSV con flujos de red, asigna una etiqueta fija (label)
    a todos los flujos, y genera un nuevo DataFrame donde cada flujo
    se asocia a una o más ventanas de tamaño 'ventana_size' segundos.
    
    Finalmente, exporta a un CSV (output_csv).
    """
    
    # 1. Leer CSV y asegurar tipo datetime
    df = pd.read_csv(input_csv)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    
    # 2. Reemplazar la columna Label con la etiqueta global (si la columna no existe, la crea)
    df["Label"] = label
    
    # 3. Diccionario donde la clave es un datetime que representa
    #    el inicio de la ventana, y el valor es una lista de flujos
    ventanas = {}

    for index, flujo in df.iterrows():
        ts_inicio = flujo["Timestamp"]
        duracion_us = flujo["Flow Duration"]  # En microsegundos (según CICFlowMeter)
        ts_fin = ts_inicio + timedelta(microseconds=duracion_us)

        # Convertir a float (segundos desde epoch)
        ts_inicio_seg = ts_inicio.timestamp()
        ts_fin_seg = ts_fin.timestamp()

        epsilon = 1e-9

        # Calcular primera y última ventana en float
        primera_ventana_seg = int(ts_inicio_seg // ventana_size) * ventana_size
        ultima_ventana_seg = int((ts_fin_seg - epsilon) // ventana_size) * ventana_size

        # Iterar sobre todas las ventanas que el flujo atraviesa
        ventana_actual = primera_ventana_seg
        while ventana_actual <= ultima_ventana_seg:
            fin_ventana = ventana_actual + ventana_size

            # Chequeamos superposición
            if ts_inicio_seg < fin_ventana and ts_fin_seg > ventana_actual:
                # Convertimos la ventana actual a datetime para usarla como clave
                ventana_key = pd.to_datetime(ventana_actual, unit='s')

                if ventana_key not in ventanas:
                    ventanas[ventana_key] = []
                ventanas[ventana_key].append(flujo)

            ventana_actual += ventana_size

    # 4. Construir un DataFrame final donde
    #    cada fila = (flujo + la ventana de 5s en la que cae)
    filas_resultantes = []

    # Ordenar las ventanas por su timestamp
    ventanas_ordenadas = sorted(ventanas.items(), key=lambda x: x[0])

    for ventana_key, lista_flujos in ventanas_ordenadas:
        for flujo_series in lista_flujos:
            flujo_dict = flujo_series.to_dict()
            # Agregamos la columna con la ventana de inicio
            flujo_dict["Ventana_Inicio"] = ventana_key
            filas_resultantes.append(flujo_dict)

    # Creamos DataFrame final
    df_result = pd.DataFrame(filas_resultantes)

    # 5. Guardamos a CSV
    df_result.to_csv(output_csv, index=False)
    print(f"¡Listo! CSV generado en: {output_csv}")
    
    return df_result

In [8]:
df_result = procesar_flujos_por_ventanas(
    input_csv="DDoS-ICMP_Fragmentation.pcap_Flow.csv", 
    label="DDoS-ICMP",         
    ventana_size=5, 
    output_csv="DDoS-ICMP_0.csv"
)

/Users/sergiomartinreizabal/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:2201: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


¡Listo! CSV generado en: DDoS-ICMP_0.csv


In [9]:
df_result[['Flow ID', 'Ventana_Inicio']].head(100)

,Flow ID,Ventana_Inicio
0,52.213.112.0-192.168.137.175-443-37193-6,2022-04-10 18:55:00
1,192.168.137.41-157.249.81.141-54332-80-6,2022-04-10 18:55:00
2,52.213.112.0-192.168.137.175-443-37193-6,2022-04-10 18:55:00
3,192.168.137.175-52.213.112.0-37193-443-6,2022-04-10 18:55:00
4,192.168.137.175-52.213.112.0-37193-443-6,2022-04-10 18:55:00
...,...,...
95,192.168.137.51-52.239.141.196-55582-443-6,2022-04-10 18:55:05
96,192.168.137.51-52.239.141.196-55581-443-6,2022-04-10 18:55:05
97,192.168.137.163-192.168.137.1-0-0-0,2022-04-10 18:55:05
98,192.168.137.163-120.76.210.199-34125-80-6,2022-04-10 18:55:05


In [10]:
# Agrupa por la columna "Ventana_Inicio" y cuenta el número de filas (flujos) en esa ventana.
conteo_flujos_por_ventana = df_result.groupby("Ventana_Inicio").size()

conteo_flujos_por_ventana.describe()

count    421.000000
mean      70.862233
std       24.459004
min        4.000000
25%       57.000000
50%       69.000000
75%       83.000000
max      208.000000
dtype: float64